# Import Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [2]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

In [3]:
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [4]:
import os
os.getcwd()

'C:\\Users\\ACHANG\\Desktop'

# Get the Toronto dataframe

In [5]:
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_wiki=pd.read_html(path)
type(df_wiki)

list

In [6]:
neighborhood=df_wiki[0]
neighborhood.rename(columns={0:'Postal Code', 1: 'Borough', 2: 'Neighborhood'}, inplace=True) # Rename the Columns
neighborhood=neighborhood.drop([0]) # remove the first row
neighborhood=neighborhood[neighborhood.Borough !='Not assigned']
neighborhood=pd.DataFrame(neighborhood)
neighborhood.set_index(['Postal Code','Borough'],inplace=True)
merge_result = neighborhood.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)
serial_wise=merge_result.reset_index()
serial_wise.loc[4, 'Neighborhood']='Queen\'s Park' # Assign the 'Borough' column value to 'Neighborhood' where 'Not assigned' occurs
serial_wise.to_excel('wikipedia_table.xls')
df=pd.DataFrame(serial_wise)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park


Input Lati/Longitute

In [7]:
df1=pd.read_csv("C:\Users\ACHANG\Desktop\input\geospatial-coordinates-toronto\Geospatial_Coordinates.csv")
input/geospatial-coordinates-toronto/Geospatial_Coordinates.csv")
df1.rename(columns={'Postal Code':'Postal Code'},inplace=True)
frames=[df,df1]
frames=pd.concat(frames, axis=1, sort=False)
merge_columns=pd.merge(df, df1, left_on='Postal Code', right_on='Postal Code')
merge_columns.to_csv('neigbors_geographical.csv')
merge_columns.head()

SyntaxError: EOL while scanning string literal (<ipython-input-7-e44d3fef1f5d>, line 2)

In [8]:
df2=pd.read_csv(r"C:\Users\ACHANG\Desktop\neigbors_geographical.csv")
df2.head()

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


dataframe cleaning

In [9]:
df3 = df2.drop(['Postcode'], axis=1)
df4 = df3.drop(['Unnamed: 0'], axis=1)
df4.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,Queen's Park,Queen's Park,43.662301,-79.389494


In [10]:
# Get the downtown Toronto data
DTT_data = df4[df4['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DTT_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,Downtown Toronto,Central Bay Street,43.657952,-79.387383


# Get the New York dataframe

In [11]:
#Input the json file from the week3
dfNY1 = pd.read_csv(r"C:\Users\ACHANG\Desktop\newyork_data.csv")
dfNY1.head()

,type,totalFeatures,features__type,features__id,features__geometry__type,features__geometry__coordinates__001,features__geometry__coordinates__002,features__geometry_name,features__properties__name,features__properties__stacked,features__properties__annoline1,features__properties__annoline2,features__properties__annoline3,features__properties__annoangle,features__properties__borough,features__properties__bbox__001,features__properties__bbox__002,features__properties__bbox__003,features__properties__bbox__004,crs__type,crs__properties__name,bbox
0,FeatureCollection,306.0,Feature,nyu_2451_34572.1,Point,-73.847201,40.894705,geom,Wakefield,1,Wakefield,NaN,NaN,0,Bronx,-73.847201,40.894705,-73.847201,40.894705,name,urn:ogc:def:crs:EPSG::4326,-74.249260
1,NaN,NaN,Feature,nyu_2451_34572.2,Point,-73.829939,40.874294,geom,Co-op City,2,Co-op,City,NaN,0,Bronx,-73.829939,40.874294,-73.829939,40.874294,NaN,NaN,40.503319
2,NaN,NaN,Feature,nyu_2451_34572.3,Point,-73.827806,40.887556,geom,Eastchester,1,Eastchester,NaN,NaN,0,Bronx,-73.827806,40.887556,-73.827806,40.887556,NaN,NaN,-73.706161
3,NaN,NaN,Feature,nyu_2451_34572.4,Point,-73.905643,40.895437,geom,Fieldston,1,Fieldston,NaN,NaN,0,Bronx,-73.905643,40.895437,-73.905643,40.895437,NaN,NaN,40.910561
4,NaN,NaN,Feature,nyu_2451_34572.5,Point,-73.912585,40.890834,geom,Riverdale,1,Riverdale,NaN,NaN,0,Bronx,-73.912585,40.890834,-73.912585,40.890834,NaN,NaN,NaN


In [12]:
# Select column
dfNY2 = dfNY1[['features__properties__borough','features__properties__annoline1', 'features__properties__bbox__002', 'features__properties__bbox__003']]
dfNY2.rename(columns={'features__properties__borough':'BoroughNY',
                          'features__properties__annoline1':'NeighborhoodNY',
                          'features__properties__bbox__002':'LatitudeNY',
                          'features__properties__bbox__003':'LongitudeNY'}, 
                 inplace=True)

dfNY2.head()

C:\Users\ACHANG\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,BoroughNY,NeighborhoodNY,LatitudeNY,LongitudeNY
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [13]:
# Data is too big, let's just use Manhattan
manhattan_data = dfNY2[dfNY2['BoroughNY'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,BoroughNY,NeighborhoodNY,LatitudeNY,LongitudeNY
0,Manhattan,Marble,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton,40.823604,-73.949688


# Foursquare API

In [14]:
CLIENT_ID = 'CVV2Y3ASWOFRBNJEEFCBYZMEXN4CTWWENIMPEIY3MBU04G12' 
CLIENT_SECRET = 'EWA0DXJBGFKYURPV5DYS0ZGVEJXMKYMMOHRSH4C0IPNJZVTY'
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:CVV2Y3ASWOFRBNJEEFCBYZMEXN4CTWWENIMPEIY3MBU04G12
CLIENT_SECRET:EWA0DXJBGFKYURPV5DYS0ZGVEJXMKYMMOHRSH4C0IPNJZVTY


In [15]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

C:\Users\ACHANG\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [16]:
# get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

C:\Users\ACHANG\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


Map of Toronto

In [17]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DTT_data['Latitude'], DTT_data['Longitude'], DTT_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [18]:
from folium import plugins
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(DTT_data['Latitude'], DTT_data['Longitude'], DTT_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

Map of Manhattan:

In [19]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['LatitudeNY'], manhattan_data['LongitudeNY'], manhattan_data['NeighborhoodNY']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [20]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

grouping = plugins.MarkerCluster().add_to(map_manhattan)

# add markers to map
for lat, lng, label in zip(manhattan_data['LatitudeNY'], manhattan_data['LongitudeNY'], manhattan_data['NeighborhoodNY']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_manhattan

# data analysis

# (1) Downtown Toronto:

In [21]:
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
downtown_toronto_venues = getNearbyVenues(names=DTT_data['Neighborhood'],
                                   latitudes=DTT_data['Latitude'],
                                   longitudes=DTT_data['Longitude'],
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [23]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(338, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Harbourfront,Regent Park",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [24]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
"Chinatown,Grange Park,Kensington Market",20,20,20,20,20,20
Christie,17,17,17,17,17,17
Church and Wellesley,20,20,20,20,20,20
"Commerce Court,Victoria Hotel",20,20,20,20,20,20


In [25]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 124 uniques categories.


In [26]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Truck,Fountain,French Restaurant,Gastropub,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [28]:
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                  venue  freq
0           Coffee Shop  0.10
1  Gym / Fitness Center  0.05
2      Sushi Restaurant  0.05
3          Concert Hall  0.05
4                 Plaza  0.05


----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.10
1      Farmers Market  0.05
2      Breakfast Spot  0.05
3            Fountain  0.05
4   French Restaurant  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


----Cabbagetown,St. James Town----
                venue  freq
0          Restaurant  0.10
1                Café  0.10
2                 Pub  0.05
3             Butcher  0.05
4  Italian Restaurant  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.30
1  Chinese Restaurant  0.05
2           Ga

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Gym / Fitness Center,Steakhouse,Opera House,Pizza Place,Plaza,Lounge,Restaurant,Concert Hall,Smoke Shop
1,Berczy Park,Seafood Restaurant,Basketball Stadium,Bakery,Park,Cocktail Bar,Museum,Coffee Shop,Breakfast Spot,Concert Hall,Restaurant
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Plane,Coffee Shop,Sculpture Garden,Boutique,Rental Car Location,Bar,Boat or Ferry
3,"Cabbagetown,St. James Town",Restaurant,Café,Indian Restaurant,Pub,Deli / Bodega,Jewelry Store,Bakery,Japanese Restaurant,Italian Restaurant,Gift Shop
4,Central Bay Street,Coffee Shop,Bar,Italian Restaurant,Chinese Restaurant,Modern European Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Ramen Restaurant,Sandwich Place,Spa
5,"Chinatown,Grange Park,Kensington Market",Café,Vietnamese Restaurant,Mexican Restaurant,Wine Bar,Bakery,Fish Market,Dessert Shop,Coffee Shop,Organic Grocery,Cocktail Bar
6,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Candy Store,Diner,Nightclub,Italian Restaurant,Restaurant
7,Church and Wellesley,Burger Joint,Pizza Place,Creperie,Park,Café,Coffee Shop,Bubble Tea Shop,Breakfast Spot,Ramen Restaurant,Mexican Restaurant
8,"Commerce Court,Victoria Hotel",Café,Coffee Shop,Bakery,Gym,Gastropub,Ice Cream Shop,Japanese Restaurant,Deli / Bodega,Museum,Pub
9,"Design Exchange,Toronto Dominion Centre",Café,Coffee Shop,Deli / Bodega,Gym / Fitness Center,Steakhouse,Pub,Restaurant,Sandwich Place,Beer Bar,Japanese Restaurant


In [31]:
# set clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 3, 2, 3, 3, 0, 3, 3])

In [32]:
downtown_toronto_merged = DTT_data
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Breakfast Spot,Distribution Center,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Spa,Bakery
1,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Café,Steakhouse,Music Venue,Burrito Place,Mexican Restaurant,Clothing Store,Plaza,Coffee Shop,Ramen Restaurant,Comic Shop
2,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Gastropub,Coffee Shop,Restaurant,Poke Place,Creperie,Middle Eastern Restaurant,Japanese Restaurant,BBQ Joint,Italian Restaurant,Café
3,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Seafood Restaurant,Basketball Stadium,Bakery,Park,Cocktail Bar,Museum,Coffee Shop,Breakfast Spot,Concert Hall,Restaurant
4,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Bar,Italian Restaurant,Chinese Restaurant,Modern European Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Ramen Restaurant,Sandwich Place,Spa


In [33]:
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [34]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront,Regent Park",Coffee Shop,Park,Breakfast Spot,Distribution Center,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Spa,Bakery
1,"Ryerson,Garden District",Café,Steakhouse,Music Venue,Burrito Place,Mexican Restaurant,Clothing Store,Plaza,Coffee Shop,Ramen Restaurant,Comic Shop
7,"Harbourfront East,Toronto Islands,Union Station",Park,Plaza,Hotel,Café,Salad Place,Skating Rink,Japanese Restaurant,Bubble Tea Shop,Sporting Goods Shop,Ice Cream Shop
11,"Chinatown,Grange Park,Kensington Market",Café,Vietnamese Restaurant,Mexican Restaurant,Wine Bar,Bakery,Fish Market,Dessert Shop,Coffee Shop,Organic Grocery,Cocktail Bar
12,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Plane,Coffee Shop,Sculpture Garden,Boutique,Rental Car Location,Bar,Boat or Ferry
16,"First Canadian Place,Underground city",Café,Coffee Shop,Restaurant,Gym / Fitness Center,Steakhouse,Gym,Gastropub,Deli / Bodega,Pizza Place,Pub
17,Church and Wellesley,Burger Joint,Pizza Place,Creperie,Park,Café,Coffee Shop,Bubble Tea Shop,Breakfast Spot,Ramen Restaurant,Mexican Restaurant


# Cluster 2 

In [35]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Stn A PO Boxes 25 The Esplanade,Cocktail Bar,Café,Farmers Market,Art Gallery,Comfort Food Restaurant,Beer Bar,Park,Jazz Club,Museum,Restaurant


# Cluster 3 

In [36]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Bay Street,Coffee Shop,Bar,Italian Restaurant,Chinese Restaurant,Modern European Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Ramen Restaurant,Sandwich Place,Spa
13,Rosedale,Park,Playground,Trail,Dance Studio,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop


# Cluster 4 

In [37]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Berczy Park,Seafood Restaurant,Basketball Stadium,Bakery,Park,Cocktail Bar,Museum,Coffee Shop,Breakfast Spot,Concert Hall,Restaurant
5,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Candy Store,Diner,Nightclub,Italian Restaurant,Restaurant
6,"Adelaide,King,Richmond",Coffee Shop,Gym / Fitness Center,Steakhouse,Opera House,Pizza Place,Plaza,Lounge,Restaurant,Concert Hall,Smoke Shop
8,"Design Exchange,Toronto Dominion Centre",Café,Coffee Shop,Deli / Bodega,Gym / Fitness Center,Steakhouse,Pub,Restaurant,Sandwich Place,Beer Bar,Japanese Restaurant
9,"Commerce Court,Victoria Hotel",Café,Coffee Shop,Bakery,Gym,Gastropub,Ice Cream Shop,Japanese Restaurant,Deli / Bodega,Museum,Pub
10,"Harbord,University of Toronto",Bookstore,Restaurant,Bakery,Japanese Restaurant,Yoga Studio,Italian Restaurant,Café,College Gym,Comfort Food Restaurant,Sandwich Place
15,"Cabbagetown,St. James Town",Restaurant,Café,Indian Restaurant,Pub,Deli / Bodega,Jewelry Store,Bakery,Japanese Restaurant,Italian Restaurant,Gift Shop


# Cluster 5 

In [38]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,St. James Town,Gastropub,Coffee Shop,Restaurant,Poke Place,Creperie,Middle Eastern Restaurant,Japanese Restaurant,BBQ Joint,Italian Restaurant,Café


# (2) Manhattan data analysis

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
manhattan_venues = getNearbyVenues(names=manhattan_data['NeighborhoodNY'],
                                   latitudes=manhattan_data['LatitudeNY'],
                                   longitudes=manhattan_data['LongitudeNY'],
                                  )

Marble
Chinatown
Washington
Inwood
Hamilton
Manhattanville
Central
East
Upper
Yorkville
Lenox
Roosevelt Island
Upper
Lincoln
Clinton
Midtown
Murray
Chelsea
Greenwich
East
Lower
Tribeca
Little
Soho
West
Manhattan
Morningside
Gramercy
Battery
Financial
Carnegie
Noho
Civic
Midtown
Sutton
Turtle
Tudor
Stuyvesant
Flatiron
Hudson


In [41]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery,20,20,20,20,20,20
Carnegie,20,20,20,20,20,20
Central,20,20,20,20,20,20
Chelsea,20,20,20,20,20,20
Chinatown,20,20,20,20,20,20
Civic,20,20,20,20,20,20
Clinton,20,20,20,20,20,20
East,40,40,40,40,40,40
Financial,20,20,20,20,20,20


In [42]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 212 uniques categories.


In [43]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Heliport,Himalayan Restaurant,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Newsstand,Noodle House,Opera House,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Tourist Information Center,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Marble,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [44]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

In [45]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery----
           venue  freq
0  Memorial Site  0.15
1           Park  0.15
2     Food Court  0.10
3       Building  0.05
4     Auditorium  0.05


----Carnegie----
                  venue  freq
0           Coffee Shop  0.10
1    Italian Restaurant  0.10
2  Gym / Fitness Center  0.10
3                   Gym  0.10
4           Karaoke Bar  0.05


----Central----
                  venue  freq
0     French Restaurant  0.10
1             Jazz Club  0.05
2                   Bar  0.05
3          Cycle Studio  0.05
4  Ethiopian Restaurant  0.05


----Chelsea----
            venue  freq
0         Theater  0.10
1  Ice Cream Shop  0.10
2    Cupcake Shop  0.05
3           Hotel  0.05
4          Market  0.05


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.10
1      Sandwich Place  0.10
2         Yoga Studio  0.05
3              Bakery  0.05
4  Spanish Restaurant  0.05


----Civic----
                  venue  freq
0     French Restaurant  0.10
1                   Spa

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery,Memorial Site,Park,Food Court,BBQ Joint,Cooking School,Playground,Performing Arts Venue,Sandwich Place,Scenic Lookout,Shopping Mall
1,Carnegie,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Gym,Pizza Place,Café,American Restaurant,Dance Studio,Bookstore,Gourmet Shop
2,Central,French Restaurant,Beer Bar,Bagel Shop,Food Truck,Music Venue,Boutique,Cocktail Bar,Ethiopian Restaurant,Library,Juice Bar
3,Chelsea,Ice Cream Shop,Theater,Taco Place,Italian Restaurant,French Restaurant,Market,Fish Market,New American Restaurant,Cupcake Shop,Coffee Shop
4,Chinatown,Sandwich Place,Chinese Restaurant,Yoga Studio,Bakery,Pizza Place,Cocktail Bar,Noodle House,Roof Deck,New American Restaurant,Museum
5,Civic,Spa,French Restaurant,Yoga Studio,General Entertainment,Molecular Gastronomy Restaurant,Falafel Restaurant,Burrito Place,Cuban Restaurant,Martial Arts Dojo,Sushi Restaurant
6,Clinton,Theater,Gym / Fitness Center,Hotel,Pizza Place,Peruvian Restaurant,Comedy Club,Coffee Shop,Cocktail Bar,Café,Building
7,East,Mexican Restaurant,Park,Vietnamese Restaurant,Dessert Shop,Thai Restaurant,Sandwich Place,Beer Store,Beer Bar,Street Art,Steakhouse
8,Financial,Coffee Shop,Gym / Fitness Center,Event Space,New American Restaurant,Salad Place,Seafood Restaurant,Monument / Landmark,Steakhouse,Jewelry Store,Restaurant
9,Flatiron,Japanese Restaurant,Yoga Studio,Coffee Shop,Russian Restaurant,Sushi Restaurant,Donut Shop,Juice Bar,Sports Club,Miscellaneous Shop,Thai Restaurant


Analysis the cluster 

In [48]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 0, 2, 3, 2, 2, 3, 1, 0, 0])

In [49]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='NeighborhoodNY')

manhattan_merged.head() 

,BoroughNY,NeighborhoodNY,LatitudeNY,LongitudeNY,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble,40.876551,-73.910660,0,Coffee Shop,Gym,Yoga Studio,Tennis Stadium,American Restaurant,Bank,Department Store,Diner,Discount Store,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,2,Sandwich Place,Chinese Restaurant,Yoga Studio,Bakery,Pizza Place,Cocktail Bar,Noodle House,Roof Deck,New American Restaurant,Museum
2,Manhattan,Washington,40.851903,-73.936900,1,Wine Shop,Park,Café,Breakfast Spot,Bakery,Pizza Place,Deli / Bodega,Ramen Restaurant,Restaurant,Coffee Shop
3,Manhattan,Inwood,40.867684,-73.921210,1,Wine Bar,Park,Café,Yoga Studio,Bakery,Deli / Bodega,Diner,Farmers Market,Mexican Restaurant,Spanish Restaurant
4,Manhattan,Hamilton,40.823604,-73.949688,0,Yoga Studio,Cocktail Bar,Mexican Restaurant,Wine Bar,Bakery,Café,Caribbean Restaurant,Mediterranean Restaurant,Historic Site,Coffee Shop


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['LatitudeNY'], manhattan_merged['LongitudeNY'], manhattan_merged['NeighborhoodNY'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Manhattan clustering

# Cluster 1

In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,NeighborhoodNY,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble,Coffee Shop,Gym,Yoga Studio,Tennis Stadium,American Restaurant,Bank,Department Store,Diner,Discount Store,Donut Shop
4,Hamilton,Yoga Studio,Cocktail Bar,Mexican Restaurant,Wine Bar,Bakery,Café,Caribbean Restaurant,Mediterranean Restaurant,Historic Site,Coffee Shop
5,Manhattanville,Italian Restaurant,Juice Bar,Ramen Restaurant,Mexican Restaurant,Seafood Restaurant,Lounge,Climbing Gym,Bike Trail,Supermarket,Dumpling Restaurant
8,Upper,Italian Restaurant,American Restaurant,Bakery,Bar,Seafood Restaurant,Bookstore,Park,Movie Theater,Juice Bar,French Restaurant
9,Yorkville,Wine Shop,Deli / Bodega,Hobby Shop,Gym / Fitness Center,Café,Liquor Store,Sandwich Place,Beer Store,Dog Run,Coffee Shop
11,Roosevelt Island,Playground,Liquor Store,Park,Outdoors & Recreation,Coffee Shop,Residential Building (Apartment / Condo),Restaurant,Deli / Bodega,Noodle House,Sandwich Place
12,Upper,Italian Restaurant,American Restaurant,Bakery,Bar,Seafood Restaurant,Bookstore,Park,Movie Theater,Juice Bar,French Restaurant
18,Greenwich,Italian Restaurant,Café,Clothing Store,Yoga Studio,Jazz Club,Food Truck,Seafood Restaurant,Caribbean Restaurant,Sandwich Place,Sushi Restaurant
20,Lower,Art Gallery,Cocktail Bar,Yoga Studio,Pet Café,Bubble Tea Shop,Café,Chinese Restaurant,Coffee Shop,Filipino Restaurant,French Restaurant
22,Little,Ice Cream Shop,Sandwich Place,Pizza Place,Gourmet Shop,Coffee Shop,Noodle House,Salad Place,Salon / Barbershop,Newsstand,French Restaurant


# Cluster 2 

In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,NeighborhoodNY,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington,Wine Shop,Park,Café,Breakfast Spot,Bakery,Pizza Place,Deli / Bodega,Ramen Restaurant,Restaurant,Coffee Shop
3,Inwood,Wine Bar,Park,Café,Yoga Studio,Bakery,Deli / Bodega,Diner,Farmers Market,Mexican Restaurant,Spanish Restaurant
7,East,Mexican Restaurant,Park,Vietnamese Restaurant,Dessert Shop,Thai Restaurant,Sandwich Place,Beer Store,Beer Bar,Street Art,Steakhouse
19,East,Mexican Restaurant,Park,Vietnamese Restaurant,Dessert Shop,Thai Restaurant,Sandwich Place,Beer Store,Beer Bar,Street Art,Steakhouse
21,Tribeca,Park,Yoga Studio,Sushi Restaurant,Hotel,Indie Theater,Italian Restaurant,Greek Restaurant,Men's Store,Cycle Studio,Poke Place
26,Morningside,Bookstore,Park,American Restaurant,Ice Cream Shop,Coffee Shop,Burger Joint,Café,Seafood Restaurant,Sandwich Place,Tennis Court
28,Battery,Memorial Site,Park,Food Court,BBQ Joint,Cooking School,Playground,Performing Arts Venue,Sandwich Place,Scenic Lookout,Shopping Mall
36,Tudor,Park,Yoga Studio,Trail,Boxing Gym,Seafood Restaurant,Spanish Restaurant,Café,Salad Place,Sushi Restaurant,Taco Place
37,Stuyvesant,Playground,Park,Boat or Ferry,Bar,Pet Service,Coffee Shop,Cocktail Bar,Farmers Market,Bistro,Gas Station


# Cluster 3 

In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,NeighborhoodNY,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sandwich Place,Chinese Restaurant,Yoga Studio,Bakery,Pizza Place,Cocktail Bar,Noodle House,Roof Deck,New American Restaurant,Museum
6,Central,French Restaurant,Beer Bar,Bagel Shop,Food Truck,Music Venue,Boutique,Cocktail Bar,Ethiopian Restaurant,Library,Juice Bar
10,Lenox,Gym,Thai Restaurant,Czech Restaurant,Salad Place,French Restaurant,Taco Place,Dessert Shop,Smoke Shop,College Academic Building,Chinese Restaurant
15,Midtown,Korean Restaurant,Hotel,Grocery Store,Japanese Restaurant,Plaza,Clothing Store,Steakhouse,Food Truck,Lingerie Store,Bookstore
16,Murray,Burger Joint,Hotel,Bagel Shop,Café,Shanghai Restaurant,Museum,French Restaurant,Sandwich Place,Sushi Restaurant,Tea Room
32,Civic,Spa,French Restaurant,Yoga Studio,General Entertainment,Molecular Gastronomy Restaurant,Falafel Restaurant,Burrito Place,Cuban Restaurant,Martial Arts Dojo,Sushi Restaurant
33,Midtown,Korean Restaurant,Hotel,Grocery Store,Japanese Restaurant,Plaza,Clothing Store,Steakhouse,Food Truck,Lingerie Store,Bookstore


# Cluster 4 

In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,NeighborhoodNY,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Clinton,Theater,Gym / Fitness Center,Hotel,Pizza Place,Peruvian Restaurant,Comedy Club,Coffee Shop,Cocktail Bar,Café,Building
17,Chelsea,Ice Cream Shop,Theater,Taco Place,Italian Restaurant,French Restaurant,Market,Fish Market,New American Restaurant,Cupcake Shop,Coffee Shop
39,Hudson,Gym / Fitness Center,Hotel,American Restaurant,Building,Furniture / Home Store,Music School,Cocktail Bar,Department Store,Residential Building (Apartment / Condo),Theater


# Cluster 5 

In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,NeighborhoodNY,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln,Theater,Performing Arts Venue,Indie Movie Theater,Concert Hall,College Arts Building,Circus,Gift Shop,Opera House,Library,Fountain
